<a href="https://colab.research.google.com/github/Suffragette/github-slideshow/blob/master/FINAL_PROJECT_evaluation_10_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null


In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]


In [3]:
!pip install pyspark==3.1.2
!pip install spark-nlp==3.3.1


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.4/212.4 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 20.0 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880756 sha256=20875e5c705af95aa76ecbd37dc891c2a6400d4fcbe59db75e007c0daa857176
  Stored in directory: /root/.cache/pip/wheels/ef/70/50/7882e1bcb5693225f7cc86698f10953201b48b3f36317c2d18
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.3/122.3 kB 3.9 MB/s eta 0:00:00


In [4]:
!java -version
!echo $JAVA_HOME


openjdk version "1.8.0_362"
OpenJDK Runtime Environment (build 1.8.0_362-8u372-ga~us1-0ubuntu1~20.04-b09)
OpenJDK 64-Bit Server VM (build 25.362-b09, mixed mode)
/usr/lib/jvm/java-8-openjdk-amd64


In [5]:
import sparknlp
spark = sparknlp.start()


In [6]:
!wget -O train_data.csv https://raw.githubusercontent.com/sebischair/Medical-Abstracts-TC-Corpus/main/medical_tc_train.csv
!wget -O test_data.csv https://raw.githubusercontent.com/sebischair/Medical-Abstracts-TC-Corpus/main/medical_tc_test.csv

train_data = spark.read.csv("train_data.csv", inferSchema=True, header=True)
test_data = spark.read.csv("test_data.csv", inferSchema=True, header=True)


--2023-07-10 07:16:34--  https://raw.githubusercontent.com/sebischair/Medical-Abstracts-TC-Corpus/main/medical_tc_train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14254503 (14M) [text/plain]
Saving to: ‘train_data.csv’

train_data.csv      100%[===================>]  13.59M  --.-KB/s    in 0.08s   

2023-07-10 07:16:35 (178 MB/s) - ‘train_data.csv’ saved [14254503/14254503]

--2023-07-10 07:16:35--  https://raw.githubusercontent.com/sebischair/Medical-Abstracts-TC-Corpus/main/medical_tc_test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting respo

In [7]:
print(train_data.columns)


['condition_label', 'medical_abstract']


In [8]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [9]:
from pyspark.ml import PipelineModel

model = PipelineModel.load("/content/gdrive/MyDrive/ΥΔΑ/ΜΑΚΡΗΣ")



In [10]:
from pyspark.ml import PipelineModel
from pyspark.sql.functions import col

# Load the trained model
model = PipelineModel.load("/content/gdrive/MyDrive/ΥΔΑ/ΜΑΚΡΗΣ")

# Check if 'bert_embeddings' column exists in the test data
if 'bert_embeddings' in test_data.columns:
    # If it exists, drop the column
    test_data = test_data.drop('bert_embeddings')

# Make predictions on the test data
predictions = model.transform(test_data)


In [11]:
print(predictions.columns)


['condition_label', 'medical_abstract', 'document', 'token', 'normalized', 'cleanTokens', 'glove_embeddings', 'bert_embeddings', 'bert_sentence_embeddings', 'class']


In [12]:
predictions.select('class').show()


+--------------------+
|               class|
+--------------------+
|[{category, 0, 15...|
|[{category, 0, 16...|
|[{category, 0, 18...|
|[{category, 0, 19...|
|[{category, 0, 68...|
|[{category, 0, 12...|
|[{category, 0, 80...|
|[{category, 0, 10...|
|[{category, 0, 22...|
|[{category, 0, 10...|
|[{category, 0, 52...|
|[{category, 0, 70...|
|[{category, 0, 98...|
|[{category, 0, 78...|
|[{category, 0, 15...|
|[{category, 0, 13...|
|[{category, 0, 14...|
|[{category, 0, 11...|
|[{category, 0, 17...|
|[{category, 0, 95...|
+--------------------+
only showing top 20 rows



In [13]:
from pyspark.sql.functions import col

# Extract the 'result' field from each dictionary in the 'class' column
predictions = predictions.withColumn('predicted_class', col('class').getItem('result'))

# Show the predicted classes
predictions.select('predicted_class').show()


+---------------+
|predicted_class|
+---------------+
|            [4]|
|            [4]|
|            [4]|
|            [4]|
|            [4]|
|            [4]|
|            [4]|
|            [4]|
|            [4]|
|            [4]|
|            [4]|
|            [4]|
|            [4]|
|            [4]|
|            [4]|
|            [4]|
|            [4]|
|            [4]|
|            [4]|
|            [4]|
+---------------+
only showing top 20 rows



In [14]:
from pyspark.sql.functions import col

# Extract the first item from each list in the 'predicted_class' column
predictions = predictions.withColumn('predicted_class', col('predicted_class').getItem(0))

# Show the predicted classes
predictions.select('predicted_class').show()


+---------------+
|predicted_class|
+---------------+
|              4|
|              4|
|              4|
|              4|
|              4|
|              4|
|              4|
|              4|
|              4|
|              4|
|              4|
|              4|
|              4|
|              4|
|              4|
|              4|
|              4|
|              4|
|              4|
|              4|
+---------------+
only showing top 20 rows



In [15]:
from sparknlp.annotator import ClassifierDLApproach


In [16]:
from sparknlp.annotator import SentenceEmbeddings
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import Tokenizer, Normalizer, StopWordsCleaner, WordEmbeddingsModel, BertEmbeddings
from pyspark.ml import Pipeline

# Define the stages of the pipeline
document_assembler = DocumentAssembler() \
    .setInputCol("medical_abstract") \
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized")

stopwords_cleaner = StopWordsCleaner() \
    .setInputCols(["normalized"]) \
    .setOutputCol("cleanTokens")

word_embeddings = WordEmbeddingsModel.pretrained() \
    .setInputCols(["document", "cleanTokens"]) \
    .setOutputCol("glove_embeddings")

bert_embeddings = BertEmbeddings.pretrained() \
    .setInputCols(["document", "cleanTokens"]) \
    .setOutputCol("bert_embeddings")

sentence_embeddings_bert = SentenceEmbeddings() \
    .setInputCols(["document", "bert_embeddings"]) \
    .setOutputCol("bert_sentence_embeddings") \
    .setPoolingStrategy("AVERAGE")

classifier = ClassifierDLApproach() \
    .setInputCols(["document", "bert_sentence_embeddings"]) \
    .setOutputCol("class") \
    .setLabelColumn("condition_label") \
    .setMaxEpochs(10) \
    .setEnableOutputLogs(True)

# Add the classifier to the pipeline
pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    normalizer,
    stopwords_cleaner,
    word_embeddings,
    bert_embeddings,
    sentence_embeddings_bert,
    classifier
])

# Fit the pipeline to the training data
model = pipeline.fit(train_data)


glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]
small_bert_L2_768 download started this may take some time.
Approximate size to download 139.6 MB
[OK!]


In [17]:
# Transform the test data
predictions = model.transform(test_data)

# Show some prediction results
predictions.select("medical_abstract", "condition_label", "class.result").show(truncate=False)


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [18]:
from pyspark.ml.tuning import ParamGridBuilder

# Define the parameter grid
paramGrid = ParamGridBuilder() \
    .addGrid(classifier.maxEpochs, [5, 10, 20]) \
    .addGrid(classifier.lr, [0.001, 0.01, 0.1]) \
    .build()

# Define the cross-validator
crossval = CrossValidator(estimator=classifier,
                          estimatorParamMaps=paramGrid,
                          evaluator=MulticlassClassificationEvaluator(labelCol="condition_label", predictionCol="prediction", metricName="accuracy"),
                          numFolds=3)  # use 3+ folds in practice


NameError: ignored

In [ ]:
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Define the cross-validator
crossval = CrossValidator(estimator=classifier,
                          estimatorParamMaps=paramGrid,
                          evaluator=MulticlassClassificationEvaluator(labelCol="condition_label", predictionCol="prediction", metricName="accuracy"),
                          numFolds=3)  # use 3+ folds in practice


In [ ]:
# Generate sentence embeddings
sentence_embeddings = BertSentenceEmbeddings.pretrained('small_bert_L12_768')\
    .setInputCols(["document"])\
    .setOutputCol("sentence_embeddings")


In [ ]:
from pyspark.sql.types import DoubleType
import pyspark.sql.functions as F
cvModel = crossval.fit(train_data)

# Convert the 'prediction' column to double type
predictions = cvModel.transform(test_data)
predictions.printSchema()
predictions = predictions.withColumn("prediction", F.col("prediction").cast(DoubleType()))

# Now you can compute the metrics
accuracy = evaluator_acc.evaluate(predictions)
f1_score = evaluator_f1.evaluate(predictions)
precision = evaluator_precision.evaluate(predictions)
recall = evaluator_recall.evaluate(predictions)

print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1_score}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")


In [ ]:
from pyspark.ml import Pipeline
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import ClassifierDLApproach, BertSentenceEmbeddings, SentenceDetectorDL
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Convert text column to Document format
documentAssembler = DocumentAssembler()\
    .setInputCol("medical_abstract")\
    .setOutputCol("document")

# Detect sentences
sentence_detector = SentenceDetectorDL.pretrained()\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

# Generate sentence embeddings
sentence_embeddings = BertSentenceEmbeddings.pretrained()\
    .setInputCols(["sentence"])\
    .setOutputCol("sentence_embeddings")

# Use ClassifierDLApproach for text classification
classifier = ClassifierDLApproach()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("prediction")\
    .setLabelColumn("condition_label")\
    .setMaxEpochs(10)\
    .setLr(0.001)

# Define the parameter grid
paramGrid = ParamGridBuilder() \
    .addGrid(classifier.maxEpochs, [5, 10, 20]) \
    .addGrid(classifier.lr, [0.001, 0.01, 0.1]) \
    .build()

# Define the cross-validator
crossval = CrossValidator(estimator=classifier,
                          estimatorParamMaps=paramGrid,
                          evaluator=MulticlassClassificationEvaluator(labelCol="condition_label", predictionCol="prediction", metricName="accuracy"),
                          numFolds=3)  # use 3+ folds in practice

# Define the pipeline
pipeline = Pipeline(stages=[documentAssembler, sentence_detector, sentence_embeddings, classifier])

# Run cross-validation, and choose the best set of parameters.
cvModel = crossval.fit(train_data)

# Make predictions on the test data
cvPredictions = cvModel.transform(test_data)

# Compute metrics on the test data
accuracy = evaluator_acc.evaluate(cvPredictions)
f1_score = evaluator_f1.evaluate(cvPredictions)
precision = evaluator_precision.evaluate(cvPredictions)
recall = evaluator_recall.evaluate(cvPredictions)

print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1_score}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")


In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql import functions as F

# Convert the prediction result from list to single value
predictions = predictions.withColumn("prediction", F.explode(F.col("class.result")))

# Initialize evaluators
evaluator_acc = MulticlassClassificationEvaluator(labelCol="condition_label", predictionCol="prediction", metricName="accuracy")
evaluator_f1 = MulticlassClassificationEvaluator(labelCol="condition_label", predictionCol="prediction", metricName="f1")
evaluator_precision = MulticlassClassificationEvaluator(labelCol="condition_label", predictionCol="prediction", metricName="weightedPrecision")
evaluator_recall = MulticlassClassificationEvaluator(labelCol="condition_label", predictionCol="prediction", metricName="weightedRecall")

# Compute metrics
accuracy = evaluator_acc.evaluate(predictions)
f1_score = evaluator_f1.evaluate(predictions)
precision = evaluator_precision.evaluate(predictions)
recall = evaluator_recall.evaluate(predictions)

print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1_score}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
